Este notebook muestra un enfoque para manejar conjuntos de datos desbalanceados utilizando el conjunto de datos MNIST como ejemplo. Primero, se cuentan el número de muestras en cada clase para identificar las clases minoritarias. Luego, se realiza un sobre-muestreo (up-sampling) para igualar el número de muestras en todas las clases utilizando la función resample de sklearn.utils. A continuación, se define un nuevo conjunto de datos equilibrado BalancedMNISTDataset que utiliza los índices de las muestras sobre-muestreadas. Finalmente, los datos equilibrados se cargan utilizando DataLoader y se itera sobre ellos.

In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.utils import resample
from torch.utils.data import DataLoader, Dataset


In [9]:
# Transformaciones de datos
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [10]:
# Descarga y carga del conjunto de datos MNIST
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

In [11]:
# Dividir el conjunto de datos en clases
class_samples = [[] for _ in range(10)]
for idx, (image, label) in enumerate(trainset):
    class_samples[label].append(idx)

In [12]:
# Contar el número de muestras en cada clase
class_counts = [len(samples) for samples in class_samples]
print("Número de muestras por clase:", class_counts)

Número de muestras por clase: [5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]


In [13]:
# Rebalanceo de clases utilizando sobre-muestreo (up-sampling)
max_samples = max(class_counts)
resampled_samples = []
for samples in class_samples:
    if len(samples) < max_samples:
        resampled_samples.extend(resample(samples, replace=True, n_samples=max_samples))
    else:
        resampled_samples.extend(samples)

In [14]:
# Definir un nuevo conjunto de datos equilibrado
class BalancedMNISTDataset(Dataset):
    def __init__(self, dataset, resampled_samples):
        self.dataset = dataset
        self.resampled_samples = resampled_samples

    def __getitem__(self, index):
        original_index = self.resampled_samples[index]
        return self.dataset[original_index]

    def __len__(self):
        return len(self.resampled_samples)


In [15]:
# Crear un nuevo conjunto de datos equilibrado
balanced_trainset = BalancedMNISTDataset(trainset, resampled_samples)

# Cargar los datos equilibrados utilizando DataLoader
batch_size = 64
trainloader = DataLoader(balanced_trainset, batch_size=batch_size, shuffle=True)

In [16]:
# Iterar sobre los datos equilibrados
for images, labels in trainloader:
    # Aquí puedes realizar el entrenamiento de tu modelo
    print("Tamaño del lote:", len(labels))
    break  # Detener después de mostrar un lote


Tamaño del lote: 64


Es importante destacar que este es solo un ejemplo básico de cómo manejar conjuntos de datos desbalanceados y que existen otros métodos y enfoques más sofisticados para tratar este problema, como sub-muestreo, técnicas de generación de muestras sintéticas, el uso de pesos de clase, entre otros. Los métodos adecuados pueden variar según el problema y es recomendable explorar más opciones según tu caso específico.